<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/fine-tuning/TinyLlama/Tech3_Finetuning_com_LoRA_e_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-tuning utilizando TinyLlama com Unsloth

### Instalação da Biblioteca `unsloth`

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

### Carregamento do modelo

Este código utiliza a biblioteca `unsloth` para carregar um modelo de linguagem quantizado em 4 bits, configurado com um comprimento máximo de sequência de 2048 tokens. O modelo é inicialmente carregado com um nome predefinido e pode ser alterado para uma versão customizada após o primeiro treino.

**Principais configurações:**
- `max_seq_length`: Comprimento máximo da sequência (2048 tokens).
- `dtype`: Detecção automática do tipo de dado. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
- `load_in_4bit`: Quantização em 4 bits ativada para economia de memória.
- `new_model_name`: Nome do modelo personalizado a ser usado após o primeiro treino.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True

# Após o 1º treino, alterar o model para rafaelpivetta/tinyllama-chat-bnb-4bit-g19
new_model_name = "unsloth/tinyllama-chat-bnb-4bit" # Alterar após o 1º treino
#new_model_name = "rafaelpivetta/tinyllama-chat-bnb-4bit-g19"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# tokenizer.pad_token = tokenizer.eos_token # pad sequences - não é indicado que o pad_token seja o mesmo que o eos_token
# tokenizer.padding_side = 'right' # right pad sequences

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

### Criação do alpaca prompt para cada linha do dataset, coluna 'text', que será usado no SFTTrainer

O código abaixo formata um conjunto de dados para treinamento de um modelo de linguagem usando um template de prompt no estilo Alpaca. O processo envolve a criação de instruções personalizadas para cada exemplo no conjunto de dados e a formatação dessas instruções em um prompt específico.

**Passos do Código:**
1. **Template `alpaca_prompt`:** Define um formato de prompt que inclui uma instrução (`Instruction`) e uma resposta (`Response`), com um token de fim de sequência (EOS_TOKEN).
2. **Função `formatting_prompts_func`:** Recebe as instruções e os conteúdos do dataset, aplica o template `alpaca_prompt`, e adiciona o token de fim de sequência, criando uma nova coluna chamada `text`.
3. **Carregamento do Dataset:** Carrega um dataset especificado pela variável `data_set_name`, utilizando a divisão de treinamento (`train`).
4. **Função `add_instruction_column`:** Cria uma nova coluna `instruction` no dataset, que consiste em uma frase que pede para criar uma descrição do produto baseado no título de cada item do dataset.
5. **Mapeamento das Funções:** Aplica as funções `add_instruction_column` e `formatting_prompts_func` ao dataset, gerando as instruções personalizadas e os textos formatados.
6. **Exemplo de Texto Formatado:** Exibe um exemplo de prompt formatado do dataset.

In [7]:
data_set_name = "LuizfvFonseca/trn_limpo_parte_1_de_70"

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # EOS_TOKEN - token de fim de sequência

# Função responsável pela criação do alpaca_prompt de cada linha e criação da coluna 'text' que será usada no treino
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["title"]
    contents     = examples["content"]
    texts = []
    for instruction, input, content in zip(instructions, inputs, contents):
        text = alpaca_prompt.format(instruction, input, content) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(data_set_name, split = "train")

def add_instruction_column(example):
    example["instruction"] =  f"Describe the product"
    return example

# Aplica a função add_instruction_column ao dataset
dataset = dataset.map(add_instruction_column)

dataset = dataset.map(formatting_prompts_func, batched = True,)

alpaca_prompt_text = dataset['text'][3]
print(alpaca_prompt_text)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
The Prophet

### Response:
In a distant, timeless place, a mysterious prophet walks the sands. At the moment of his departure, he wishes to offer the people gifts but possesses nothing. The people gather round, each asks a question of the heart, and the man's wisdom is his gift. It is Gibran's gift to us, as well, for Gibran's prophet is rivaled in his wisdom only by the founders of the world's great religions. On the most basic topics--marriage, children, friendship, work, pleasure--his words have a power and lucidity that in another era would surely have provoked the description "divinely inspired." Free of dogma, free of power structures and metaphysics, consider these poetic, moving aphorisms a 20th-century supplement to all sacred traditions--as millions of other readers alre

### Inferência antes do treino

1. **Habilita Inferência Rápida:** O método `FastLanguageModel.for_inference(model)` é usado para ativar uma inferência até 2 vezes mais rápida no modelo carregado.
2. **Tokenização dos Inputs:** A instrução é formatada com o `alpaca_prompt` para criar um prompt, que é então tokenizado e preparado para ser processado pelo modelo em um dispositivo CUDA (GPU).

In [8]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product",
        "The Prophet", # instruction
        "", # response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

print(inputs)

{'input_ids': tensor([[    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29892,
          3300,  2859,   411,   385,  1881,   393,  8128,  4340,  3030, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  4002,
         29581,   278,  3234,    13,    13,  2277, 29937, 10567, 29901,    13,
          1576,  1019,   561,   300,    13,    13,  2277, 29937, 13291, 29901,
            13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


3. **Geração de Texto:** Geração de saída com base nos inputs, limitando a geração a 128 tokens. A variável `use_cache` é setada para True para otimizar a geração.

In [9]:
outputs = model.generate(**inputs,
                         max_new_tokens = 128,
                        #  use_cache = True,
                        #  temperature=0.7,
                        #  top_p=0.9,
                        #  repetition_penalty=1.1
                         )

print(outputs)

tensor([[    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29892,
          3300,  2859,   411,   385,  1881,   393,  8128,  4340,  3030, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  4002,
         29581,   278,  3234,    13,    13,  2277, 29937, 10567, 29901,    13,
          1576,  1019,   561,   300,    13,    13,  2277, 29937, 13291, 29901,
            13,  1576,  1019,   561,   300,   338,   263, 13988,   322,  1224,
         24285,  3234,   393,   508,   367,  1304,   363,   263,  9377,  3464,
           310,  8324, 29889,   739,   338,  8688,   304,  3867,  1880, 29899,
         29567,  6047,   322,  3682,   284, 10348,  4180, 29892,  3907,   372,
         10839,   363,   671,   297,   263, 12875,   310,  6055, 29892,  3704,
          4696,  5802, 29892, 16867, 29892,   322,  5735, 21637, 29889,   450,
          1019,   561,   300,  5680,   263,  3464,  

4. **Decodificação da Saída:** Os tokens gerados são decodificados de volta para texto, resultando na resposta gerada pelo modelo para a instrução dada.

In [10]:
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe the product\n\n### Input:\nThe Prophet\n\n### Response:\nThe Prophet is a powerful and versatile product that can be used for a wide range of applications. It is designed to provide high-quality sound and exceptional audio performance, making it ideal for use in a variety of settings, including music production, recording, and live performances. The Prophet features a range of features, including a 100-watt power amplifier, 12-inch Neodymium drivers, and a 10-inch Neodymium driver, which provide clear and dynamic sound with excellent detail and resolution. The Prophet also includes a high-performance']

### Configuração do Modelo com PEFT (Parameter-Efficient Fine-Tuning)

Uso da função `get_peft_model` da biblioteca `FastLanguageModel` para aplicar o fine-tuning eficiente de parâmetros (PEFT) a um modelo de linguagem. As configurações incluem a aplicação da técnica LoRA (Low-Rank Adaptation) para ajustar os parâmetros de forma eficiente, utilizando menos memória e possibilitando o treinamento de lotes maiores.

**Principais Configurações:**
- **Rank `r`:** Define a dimensão do espaço de rank baixo (exemplo: 16).
- **Módulos Alvo (`target_modules`):** Lista de módulos específicos do modelo para aplicar a adaptação LoRA, como projeções de queries (`q_proj`), chaves (`k_proj`), valores (`v_proj`), entre outros.
- **`lora_alpha`:** Parâmetro de escala para o ajuste LoRA.
- **`lora_dropout`:** Probabilidade de dropout aplicada nos caminhos LoRA. 0 é otimizado para essa configuração.
- **`bias`:** Configuração de bias, com a opção "none" sendo a mais otimizada.
- **`use_gradient_checkpointing`:** Ativado como "unsloth", que permite economizar 30% de VRAM e suportar contextos muito longos com batch sizes maiores.
- **`use_rslora`:** Controle de ativação para Rank Stabilized LoRA, uma variação da técnica.
- **`loftq_config`:** Configuração para LoftQ, que não está sendo utilizada neste exemplo.

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj","lm_head"], # Teste com todos os parâmetros
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 é otimizado, mas gera risco de overfitting
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post3 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Garbage Collection e Limpeza de Cache

Indicado para otimizar o uso de recursos durante o treinamento de modelos no Google Colab. A coleta de lixo ajuda a liberar memória ocupada por objetos que não são mais necessários, enquanto a limpeza do cache do CUDA assegura que a memória da GPU seja liberada.

In [12]:
#Garbage collection para que os recursos no colab não excedam no momento do treino
import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() #clean cache

## Configuração do SFTTrainer para Treinamento de Modelos

Este trecho configura um `SFTTrainer` (Supervised Fine-Tuning Trainer) para treinar um modelo de linguagem utilizando a biblioteca `transformers` e a biblioteca `unsloth`. O `SFTTrainer` é utilizado para realizar o fine-tuning de modelos de forma eficiente.

**Principais Componentes da Configuração:**

- **Modelo e Tokenizer:** O modelo e o tokenizer já carregados são passados para o treinador.
- **Dataset de Treinamento:** O dataset utilizado para treinamento é especificado, incluindo o campo de texto (`dataset_text_field`).
- **Max Sequence Length:** Define o comprimento máximo da sequência para o treinamento.

**Parâmetros do Treinamento:**
- **Batch Size:** `per_device_train_batch_size` é definido como 2, especificando quantos exemplos serão processados em cada passo de treinamento.
- **Gradient Accumulation:** `gradient_accumulation_steps` é definido como 16, permitindo acumular gradientes antes de realizar uma atualização do modelo.
- **Warmup Steps:** Define o número de passos de aquecimento antes de aumentar a taxa de aprendizado.
- **Max Steps:** O treinamento será limitado a 100 passos.
- **Learning Rate:** A taxa de aprendizado é definida como 2e-4.
- **Mixed Precision:** `fp16` e `bf16` são utilizados dependendo do suporte para `bfloat16`.
- **Gradient Clipping:** `max_grad_norm` é definido como 0.3 para estabilizar o treinamento.
- **Logging:** Define que as informações de log serão exibidas a cada passo.
- **Otimização:** Utiliza o otimizador "adamw_8bit" com uma taxa de decaimento de peso de 0.01.
- **Scheduler:** O tipo de agendador de taxa de aprendizado é definido como "linear".
- **Seed:** Um valor de semente para garantir a reprodutibilidade.
- **Group by Length:** Agrupa sequências de comprimentos semelhantes para melhorar a eficiência do treinamento.
- **Output Directory:** Especifica o diretório de saída para salvar os resultados.

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # pode tornar o treinamento 5 vezes mais rápido para sequências curtas.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # indica que o modelo deve passar por todo o conjunto de dados de treinamento apenas uma vez
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        max_grad_norm=0.6, # previne que os gradientes fiquem muito grandes e ajuda a estabilizar o treinamento. Gradient Clipping
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        group_by_length=True, # agrupa sequências de comprimento semelhante para tornar o treinamento mais eficiente.
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/20000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### Treinamento do modelo

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 60
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
1,2.256700
2,2.185000
3,2.240000
4,2.143400
5,2.328000
6,2.387600
7,2.341500
8,2.388900
9,2.226300
10,2.244100


### Inferência após o treino

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product",
        "The Prophet", # instruction
        "", # response - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs,
                         max_new_tokens = 128,
                         #use_cache = True,
                         #temperature=0.7,
                         #top_p=0.9,
                         #repetition_penalty=1.1
                         )
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe the product\n\n### Input:\nThe Prophet\n\n### Response:\n"The Prophet is a masterpiece of the Islamic tradition, a work of art that has been read and admired by millions of Muslims and non-Muslims alike."--Mohammed Yunus, Nobel Peace Prize Laureate</s>']

### Inferência após o treino utilizando `TextStreamer`

In [16]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product", # instruction
        "The Prophet",
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, )

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
The Prophet

### Response:
"The Prophet is a masterpiece of the Islamic tradition, a work of art that has been read and admired by millions of Muslims and non-Muslims alike."--Mohammed Yunus, Nobel Peace Prize Laureate</s>


### Carregamento do modelo finetuned para o HuggingFace

In [ ]:
from google.colab import userdata

# import gc # garbage collection
# gc.collect()
# torch.cuda.empty_cache() #clean cache

#model.save_pretrained("lora_model") # Local saving
#tokenizer.save_pretrained("lora_model")
model.push_to_hub(new_model_name, token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub(new_model_name, token = userdata.get('HF_TOKEN')) # Online saving


README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/50.5M [00:00<?, ?B/s]

Saved model to https://huggingface.co/rafaelpivetta/tinyllama-chat-bnb-4bit-g19


No files have been modified since last commit. Skipping to prevent empty commit.


### Fontes

- [Databricks Blog: Efficient Fine-Tuning with LoRA](https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms)
- [GitHub - Unsloth](https://github.com/unslothai/unsloth?tab=readme-ov-file)
- [LoRA — Intuitively and Exhaustively Explained](https://medium.com/p/e944a6bff46b)
